### creating agents

In [123]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [124]:
api_wraper =WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=1000)
wiki = WikipediaQueryRun(api_wrapper=api_wraper)

In [125]:
wiki.name

'wikipedia'

In [164]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults


wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=4)
ddg_search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")

In [165]:
ddg_search.run("machine learning history")

"[snippet: The fusion of machine learning (ML) and analytics has brought about a profound revolution in the healthcare sector. This transformation has ushered in innovative solutions aimed at enhancing patient outcomes, streamlining operational processes, and mitigating costs. This paper provides an insight of the predominant themes and discoveries within ..., title: Analytics of Machine Learning in Healthcare Industries, link: https://link.springer.com/chapter/10.1007/978-981-97-1329-5_1], [snippet: The history of brain-computer interfaces (BCIs) starts with Hans Berger's discovery of the brain's electrical activity and the development of electroencephalography (EEG). In 1924 Berger was the first to record human brain activity by means of EEG. ... Advanced machine learning methods were recently developed to compute a subject-specific ..., title: Brain-computer interface - Wikipedia, link: https://en.wikipedia.org/wiki/Brain–computer_interface], [snippet: Abstract Ancient manuscripts s

In [166]:
# arxiv wraper
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun




In [167]:
arxiv_wraper =ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=api_wraper)

In [168]:
tools = [ddg_search,arxiv]

In [169]:
import os 
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

In [170]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [171]:
prompt = f"""
As an experienced blog writer, generate a blog based on the given topic and infomation. Ensure the blog follows this structure:

Heading:
Clearly define the topic of the blog.

Introduction:
Provide an engaging introduction to the topic.

Content:
Present detailed and informative content, supported by research and relevant sources.

Summary:
Summarize the main points covered in the blog.

from given infomation only
"""

In [191]:
from langchain import hub
prompt = hub.pull("riphunter7001x/custom-template-for-blog-generation")

In [192]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'topic'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'riphunter7001x', 'lc_hub_repo': 'custom-template-for-blog-generation', 'lc_hub_commit_hash': '10ae18007884fdc5a05a82e0b216504b5973c9610055f78f33bb06bb3b11e6fc'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='As an experienced blog writer, generate a blog based on the given topic and infomation. Ensure the blog follows this structure:\nHeading:\nClearly define the topic of the blog.\nIntroduction:\nProvide an engaging introduction to the topic.\nContent:\nPresent detailed and informative content, supported by research and relevant sour

In [193]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [194]:
# agent execiter 
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent = agent,tools=tools,verbose =True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'topic'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'riphunter7001x', 'lc_hub_repo': 'custom-template-for-blog-generation', 'lc_hub_commit_hash': '10ae18007884fdc5a05a82e0b216504b5973c9610055f78f33bb06bb3b11e6fc'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='As an experienced blog writer, generate a blog based on the given topic and infomation. Ensure the blog follows this structure:\nHead

In [195]:
response = agent_executer.invoke({"topic":"isreael-pelestine war"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'Israel-Palestine war'}`


[snippet: Israel-Gaza war. The Israel-Gaza war has gone on for six months, and tensions have spilled into the surrounding region.. The war: On Oct. 7, Hamas militants launched an unprecedented cross-border ..., title: Latest Israel-Hamas war news and Gaza conflict updates - The Washington ..., link: https://www.washingtonpost.com/world/2024/05/15/israel-hamas-war-news-gaza-palestine-rafah/], [snippet: Palestinians on Wednesday mark the 76th year of their mass expulsion from what is now Israel, with many fearing the war on Gaza repeats their painful history on an even more cataclysmic scale., title: Israel's war on Gaza live: New Nakba as hundreds of thousands flee ..., link: https://www.aljazeera.com/news/liveblog/2024/5/15/israels-war-on-gaza-live-new-nakba-as-hundreds-of-thousands-flee-attacks], [snippet: Israel and Hamas have been waging war since gunmen from the P

In [196]:
print(response["output"])

### Topic: Israel-Palestine War

#### Introduction:
The Israel-Palestine conflict, often referred to as the Israel-Palestine war, is a long-standing and complex dispute between the State of Israel and the Palestinian people. This conflict has deep historical roots and has resulted in numerous wars, violence, and ongoing tensions in the region. Understanding the background, causes, and current status of the Israel-Palestine war is crucial to grasp the complexities of the situation.

#### Content:
The Israel-Palestine conflict dates back to the late 19th and early 20th centuries when Jewish immigration to Palestine increased due to the Zionist movement, which sought to establish a Jewish homeland in the region. The conflict escalated with the establishment of the State of Israel in 1948, leading to the displacement of hundreds of thousands of Palestinians, known as the Nakba or "catastrophe."

Over the years, several wars and conflicts have taken place between Israel and the Palestinians

In [197]:
response = agent_executer.invoke({"topic":"DARPA( Defense Advanced Research Projects Agency)"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'DARPA'}`


[snippet: The DARPA THREADS program aims to overcome the temperature limits at the transistor scale inherent to power-amplifying functions. With new materials and approaches to diffusing the heat that degrades performance and mission life for monolithic microwave integrated circuits (MMICs), THREADS aims to resolve the thermal management challenges of ..., title: BAE Systems awarded $12 million to boost effectiveness of radio ..., link: https://www.defence-industries.com/news/bae-systems-awarded-to-boost-effectiveness-of-radio-frequency-based-applications], [snippet: News: Microelectronics 15 May 2024. BAE Systems awarded $12m THREADS contract to boost effectiveness of RF-based applications. The Defense Advanced Research Projects Agency (DARPA) has awarded BAE Systems' FAST Labs R&D organization in Merrimack, NH, USA a $12m contract for the program THREADS (Technologies for Heat Rem

In [198]:
print(response["output"])

## DARPA: Innovations and Impact

### Introduction
The Defense Advanced Research Projects Agency (DARPA) is a renowned agency known for its cutting-edge research and development projects that have significantly impacted various fields. From technological advancements to scientific breakthroughs, DARPA has been at the forefront of innovation. In this blog, we will delve into the history, achievements, and ongoing projects of DARPA to understand its profound influence on the world.

### History of DARPA
Established in 1958 in response to the Soviet Union's launch of Sputnik, DARPA was created with the mission to prevent technological surprises and maintain the technological superiority of the U.S. military. Over the years, DARPA has been instrumental in pioneering groundbreaking technologies that have revolutionized defense, communication, and science.

### Achievements and Impact
#### 1. Technological Innovations
DARPA has been responsible for numerous technological innovations that hav

In [199]:
response = agent_executer.invoke({"topic":"attention all you need"})




> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention: All You Need'}`
responded: I will now write a blog on the topic "Attention: All You Need."

Published: 2023-03-02
Title: Methods used in nanostructure modeling
Authors: I. Camps
Summary: How many times you need to change your method description because you were
"accused" of plagiarism from text you already published? I will use this
preprint to add all the methods I currently used in running the simulations for
my research works. Then, I will cite it as needed.# Attention: All You Need

## Introduction
In the realm of artificial intelligence and machine learning, the concept of attention mechanisms has gained significant attention (pun intended) in recent years. The introduction of the "Attention Is All You Need" paper by Vaswani et al. in 2017 revolutionized the field of natural language processing by introducing the Transformer model, which relies solely on attention mechanisms for feature extracti

In [200]:
print(response["output"])

# Attention: All You Need

## Introduction
In the realm of artificial intelligence and machine learning, the concept of attention mechanisms has gained significant attention (pun intended) in recent years. The introduction of the "Attention Is All You Need" paper by Vaswani et al. in 2017 revolutionized the field of natural language processing by introducing the Transformer model, which relies solely on attention mechanisms for feature extraction and sequence modeling. This blog delves into the significance of attention mechanisms, their applications, and their impact on various domains.

## Content
### Understanding Attention Mechanisms
Attention mechanisms in machine learning mimic the human cognitive process of selectively focusing on specific parts of input data while ignoring others. In the context of neural networks, attention mechanisms enable models to weigh the importance of different input elements when making predictions or generating outputs. Unlike traditional sequence mod

In [201]:
response = agent_executer.invoke({"topic":"Training Compute-Optimal LLM"})




> Entering new AgentExecutor chain...
# Training Compute-Optimal Large Language Models

## Introduction
Large Language Models (LLMs) have revolutionized natural language processing tasks by achieving state-of-the-art performance in various language-related tasks. However, training these models requires significant computational resources and time. In this blog, we will delve into the concept of training compute-optimal LLMs, exploring techniques and strategies to optimize the training process for these complex models.

## Content

### 1. Understanding Large Language Models (LLMs)
Large Language Models are deep learning models that process and generate human language. They have been instrumental in tasks like language translation, text generation, and sentiment analysis. Examples of popular LLMs include GPT-3 (Generative Pre-trained Transformer 3) and BERT (Bidirectional Encoder Representations from Transformers).

### 2. Challenges in Training LLMs
Training LLMs poses several challen

In [202]:
print(response["output"])

# Training Compute-Optimal Large Language Models

## Introduction
Large Language Models (LLMs) have revolutionized natural language processing tasks by achieving state-of-the-art performance in various language-related tasks. However, training these models requires significant computational resources and time. In this blog, we will delve into the concept of training compute-optimal LLMs, exploring techniques and strategies to optimize the training process for these complex models.

## Content

### 1. Understanding Large Language Models (LLMs)
Large Language Models are deep learning models that process and generate human language. They have been instrumental in tasks like language translation, text generation, and sentiment analysis. Examples of popular LLMs include GPT-3 (Generative Pre-trained Transformer 3) and BERT (Bidirectional Encoder Representations from Transformers).

### 2. Challenges in Training LLMs
Training LLMs poses several challenges due to their size and complexity. Th